In [1]:
import urllib2
from bs4 import BeautifulSoup as BS
import pandas as pd

In [2]:
def generate_song_list():
    # returns random list of links to song's webpage on tanzmusik-online
    
    # get main webpage 
    music = "https://www.tanzmusik-online.de/random"
    music_page = urllib2.urlopen(music)
    music_soup = BS(music_page)
    # get the songs listed
    song_list=music_soup.find("div", class_="songlist")
    # get all links to song titles
    all_songs = song_list.find_all("div", class_="songTitle")
    links = []
    for song in all_songs:
        link = song.find("a").get("href")
        links.append(link)
    return links

In [3]:
def get_label_and_youtube_link(song):
    # returns the specified dance style as label and the youtube link to the song

    # get the song's webpage
    song_page = urllib2.urlopen(song)
    song_soup = BS(song_page)
    # get the links to associated pages
    all_modes = song_soup.find_all("span", class_="modelink")
    for mode in all_modes:
        link = mode.find("a").get("href")
        # only continue if associated page is youtube
        if "youtube" in link:
            labels = []
            # get the assigned dances, might either be multiple divs
            # or multiple links in a single div
            all_dances = song_soup.find_all("div", class_="dances")
            for ad in all_dances:
                dances = ad.find_all("a")
                for dance in dances:
                    labels.append(dance.string)
            # since we expect only one youtube link, we can return here
            return link, labels
    # in case no youtube link was found, we will not return any relevant info

In [4]:
# generate dance music samples
links = []
for i in range(100):
    links.extend(generate_song_list())
# remove any duplicates
links = set(links)

In [16]:
# get youtubelinks and dance type labels
song_list = []
for song in links:
    # for each song get the labels and a youtube link
    return_values = get_label_and_youtube_link(song)
    # if relevant information is provided
    if return_values is not None:
        # for each label add a row with the youtube link
        for irv in range(len(return_values[1])):
            song_list.append([return_values[0], return_values[1][irv]])
labels = pd.DataFrame(song_list, columns=["youtube_link", "label"])

URLError: <urlopen error [Errno -2] Name or service not known>

In [33]:
# put id and youtube link into separate data frame to be able to add relevant information later
song_info = pd.DataFrame(labels["youtube_link"].unique(), columns=["youtube_link"])
song_info["id"] = song_info["youtube_link"].str.split("=").str[1]

In [37]:
# instead of link, save id with labels, only
labels["id"] = labels["youtube_link"].str.split("=").str[1]
labels = labels.drop("youtube_link", axis=1)

In [38]:
# save lists
labels.to_csv("labels_list.csv")
song_info.to_csv("song_info_list.csv")